In [1]:
from pymongo import MongoClient
# from sshtunnel import SSHTunnelForwarder
import pymongo
from tqdm import tqdm
import os
import re

import numpy as np
import pandas as pd

import dateutil.parser

In [2]:
# 使用GPU
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
! nvidia-smi

Sun Jun 12 14:33:07 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.129.06   Driver Version: 470.129.06   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| 30%   38C    P8    21W / 260W |     22MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:02:00.0 Off |                  N/A |
| 28%   

# MongoDB

In [4]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

try:
    conn = pymongo.MongoClient(mongoURI)
    db = conn.eva
    # db_de = db.patent_de
    # db_us = db.patent_us
    # db_cn = db.patent_cn
    
    db_rm_us = db.rm_patent_us
    db_rm_cn = db.rm_patent_cn
    db_rm_de = db.rm_patent_de

    db_rm_us_2011 = db.rm_patent_us_2011
    db_rm_cn_2011 = db.rm_patent_cn_2011
    db_rm_de_2011 = db.rm_patent_de_2011
    
except errors.ConnectionFailure as err:
    print(err)

In [5]:
db_rm_us_2011

Collection(Database(MongoClient(host=['140.117.69.70:30241'], document_class=dict, tz_aware=False, connect=True, authmechanism='SCRAM-SHA-1'), 'eva'), 'rm_patent_us_2011')

# US-DE-CN

## 2011

### DE

In [6]:
data_assignee = db_rm_de_2011.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}
    # {"$limit":10}
])

ls_company = []
ls_count = []
for data in data_assignee:
    #print(data["_id"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])

In [7]:
len(ls_company)

0

In [8]:
ls_c = []
ls_family = []
ls_count = []

for c in ls_company:
    data_fam = db_rm_de_2011.aggregate([
        {"$match" : {"current_assignee":{"$regex":c}}},
        {"$unwind" : "$current_assignee"},
        {"$unwind" : "$countrystatus"},
        {"$group" : {"_id" : "$countrystatus", "count":{"$sum":1}}},
        {"$sort": {"count":-1}}
        # {"$limit":5}
    ])
    

    for data in data_fam:
        #print(data)
        ls_c.append(c)
        ls_family.append(data["_id"])
        ls_count.append(data["count"])
        
df_family_de = pd.DataFrame((zip(ls_c, ls_family, ls_count)), columns = ['company', 'country', 'count'])
df_family_de.head()

,company,country,count


In [ ]:
df_family_de.shape

(160, 3)

In [ ]:
df_family_de.drop(df_family_de[df_family_de["country"]=="WO"].index, inplace=True)
df_family_de.reset_index(drop=True, inplace=True)

In [ ]:
# 轉成company-country 的總數表格
table_de = df_family_de.pivot_table(index='company', values='count', columns='country').fillna(0).reset_index()
c_ls_de = [c for c in table_de.columns if re.search('[A-Za-z]', c)]
final_table_de = table_de[c_ls_de]
final_table_de.head()

country,company,AT,BR,CA,CH,CN,DE,DK,EP,ES,...,JP,KR,LT,PL,PT,RS,SG,TW,US,WO
0,Advantech Service IoT GmbH,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
1,Apple Inc,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
2,Armstrong DLW GmbH,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Audi AG,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,DLOG GES fur ELEKTRONISCHE DATENTECHNIK MBH,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0


In [ ]:
final_table_de.shape

(41, 24)

In [ ]:
final_table_de.to_csv('rm_company_country_de_2011.csv',index=0)

final_table存進MongoDB

In [ ]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

conn = pymongo.MongoClient(mongoURI)
db = conn.eva
conn_db_collection = db.rm_company_country_de_2011

In [ ]:
# title
conn_db_collection.insert_one({"company":final_table_de.columns.tolist()[0],"country":final_table_de.columns.tolist()[1:]})

# 內容
for k in final_table_de.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    conn_db_collection.insert_one({"company":k[0],"country":k[1:]})

### CN

In [9]:
data_assignee = db_rm_cn_2011.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}
    # {"$limit":10}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])

In [10]:
len(ls_company)

386

In [11]:
ls_c = []
ls_family = []
ls_count = []

for c in ls_company:
    data_fam = db_rm_cn_2011.aggregate([
        {"$match" : {"current_assignee":{"$regex":c}}},
        {"$unwind" : "$current_assignee"},
        {"$unwind" : "$countrystatus"},
        {"$group" : {"_id" : "$countrystatus", "count":{"$sum":1}}},
        {"$sort": {"count":-1}}
        # {"$limit":5}
    ])
    

    for data in data_fam:
        #print(data)
        ls_c.append(c)
        ls_family.append(data["_id"])
        ls_count.append(data["count"])
        
df_family_cn = pd.DataFrame((zip(ls_c, ls_family, ls_count)), columns = ['company', 'country', 'count'])
df_family_cn.head()

,company,country,count
0,"Jinhu Zhongbo IOT Technology Co., Ltd.",CN,30
1,Jiangsu IoT Research and Development Center,CN,17
2,"Gaoxing Zhilian Technology Co., Ltd.",CN,9
3,"Gaoxing Zhilian Technology Co., Ltd.",WO,7
4,China core Microelectronics Technology Chengdu...,CN,7


In [12]:
df_family_cn.shape

(1188, 3)

In [13]:
df_family_cn.drop(df_family_cn[df_family_cn["country"]=="WO"].index, inplace=True)
df_family_cn.reset_index(drop=True, inplace=True)

In [14]:
# 轉成company-country 的總數表格
table_cn = df_family_cn.pivot_table(index='company', values='count', columns='country').fillna(0).reset_index()
c_ls_cn = [c for c in table_cn.columns if re.search('[A-Za-z]', c)]
final_table_cn = table_cn[c_ls_cn]
final_table_cn.head()

country,company,AP,AR,AU,BR,CA,CL,CN,CO,CR,...,RU,SG,SI,SM,TR,TW,UA,US,UY,ZA
0,ABB Grid Switzerland AG,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
1,AVIC Huadong Photoelectric Co Ltd,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Acclarent Inc,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,Acer Inc,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Active Semi International Inc USA,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
final_table_cn.shape

(379, 52)

In [16]:
final_table_cn.to_csv('rm_company_country_cn_2011.csv',index=0)

final_table存進MongoDB

In [17]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

conn = pymongo.MongoClient(mongoURI)
db = conn.eva
conn_db_collection = db.rm_company_country_cn_2011

In [18]:
# title
conn_db_collection.insert_one({"company":final_table_cn.columns.tolist()[0],"country":final_table_cn.columns.tolist()[1:]})

# 內容
for k in final_table_cn.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    conn_db_collection.insert_one({"company":k[0],"country":k[1:]})

### US

In [19]:
data_assignee = db_rm_us_2011.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}#,
    #{"$limit":10}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"])
    ls_company.append(data["_id"].replace("))",")"))
    ls_count.append(data["count"])

In [20]:
len(ls_company)

52

In [21]:
ls_c = []
ls_family = []
ls_count = []

for c in ls_company:
    try:
        data_fam = db_rm_us_2011.aggregate([
        {"$match" : {"current_assignee":{"$regex":c}}},
        {"$unwind" : "$current_assignee"},
        {"$unwind" : "$countrystatus"},
        {"$group" : {"_id" : "$countrystatus", "count":{"$sum":1}}},
        {"$sort": {"count":-1}}
        # {"$limit":5}
    ])
    except:
        print(c)
    

    for data in data_fam:
        #print(data)
        ls_c.append(c)
        ls_family.append(data["_id"])
        ls_count.append(data["count"])
        
df_family_us = pd.DataFrame((zip(ls_c, ls_family, ls_count)), columns = ['company', 'country', 'count'])
df_family_us.head()

,company,country,count
0,Qualcomm Inc,WO,14
1,Qualcomm Inc,US,14
2,Qualcomm Inc,CN,13
3,Qualcomm Inc,EP,13
4,Qualcomm Inc,JP,13


In [22]:
df_family_us.shape

(205, 3)

In [23]:
df_family_us.drop(df_family_us[df_family_us["country"]=="WO"].index, inplace=True)
df_family_us.reset_index(drop=True, inplace=True)

In [24]:
# 轉成company-country 的總數表格
table_us = df_family_us.pivot_table(index='company', values='count', columns='country').fillna(0).reset_index()
c_ls_us = [c for c in table_us.columns if re.search('[A-Za-z]', c)]
final_table_us = table_us[c_ls_us]
final_table_us.head()

country,company,AU,BR,CA,CN,DE,EP,ES,GB,HK,HU,IN,JP,KR,MX,PL,RU,SG,TW,US
0,Affirmed Networks Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Alcatel Lucent SAS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,Apple Inc,0.0,1.0,0.0,2.0,0.0,2.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,3.0
3,Atec Iot Co Ltd,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0
4,BORQS WIRELESS Ltd,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [25]:
final_table_us.shape

(52, 20)

In [26]:
final_table_us.to_csv('rm_company_country_us_2011.csv',index=0)

final_table存進MongoDB

In [27]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

conn = pymongo.MongoClient(mongoURI)
db = conn.eva
conn_db_collection = db.rm_company_country_us_2011

In [28]:
# title
conn_db_collection.insert_one({"company":final_table_us.columns.tolist()[0],"country":final_table_us.columns.tolist()[1:]})

# 內容
for k in final_table_us.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    conn_db_collection.insert_one({"company":k[0],"country":k[1:]})

### 合併

In [29]:
# concat de cn
# df_c = pd.concat([final_table_de,final_table_cn], join='outer')
df_c2 = pd.concat([final_table_cn,final_table_us], join='outer')
df_c2 = df_c2.fillna(0)
#reset index
t = list(df_c2.sort_index(axis=1).columns)
df_c2 = df_c2.reindex(columns=[t[-1]]+t[:-1])
df_c2

country,company,AP,AR,AU,BR,CA,CL,CN,CO,CR,...,RU,SG,SI,SM,TR,TW,UA,US,UY,ZA
0,ABB Grid Switzerland AG,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
1,AVIC Huadong Photoelectric Co Ltd,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Acclarent Inc,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,Acer Inc,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Active Semi International Inc USA,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47,Toray Plastics America Inc,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
48,US Department of Navy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
49,WSOU Investments LLC,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
50,Xerox Corp,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,0.0,0.0


In [30]:
# 合併重複的公司
df_c2 = df_c2.groupby(df_c2.company, axis=0).sum().reset_index()

In [31]:
df_c2.head()

country,company,AP,AR,AU,BR,CA,CL,CN,CO,CR,...,RU,SG,SI,SM,TR,TW,UA,US,UY,ZA
0,ABB Grid Switzerland AG,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
1,AVIC Huadong Photoelectric Co Ltd,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Acclarent Inc,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,Acer Inc,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Active Semi International Inc USA,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
df_c2.to_csv('rm_company_country_all_2011.csv',index=0)

In [33]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

conn = pymongo.MongoClient(mongoURI)
db = conn.eva
conn_db_collection = db.rm_company_country_all_2011

In [34]:
# dict_de = {k[0]:k[1:] for k in final_table_de.values.tolist()}

In [35]:
# title
conn_db_collection.insert_one({"company":df_c2.columns.tolist()[0],"country":df_c2.columns.tolist()[1:]})

# 內容
for k in df_c2.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    conn_db_collection.insert_one({"company":k[0],"country":k[1:]})